# Imports

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import os

import numpy as np
import pandas as pd

In [57]:
sys.path.append(os.getcwd() + '/refi')
import retirement_analysis as ra
import constants
#import asset
#import consumption
#import inflation
import portfolio as pf

import series.base_series as base_series
from series.consumption import StaticRetirementConsumption
from series.deferral import StaticPreRetirementDeferral

# Constants

In [6]:
initial_balance = 400000
initial_cpi = 275
initial_age = 30
initial_deferral = 19500
retirement_age = 60
retirement_consumption = 75000
death_age = 100

In [7]:
num_years = death_age - initial_age + 1

In [8]:
geo_mean_equity_return = .065
geo_mean_bond_return = .035
geo_mean_cash_return = .002

geo_mean_inflation = .02


# Portfolio

In [9]:
equity = base_series.StaticSeries(num_years, geo_mean_equity_return)
bond = base_series.StaticSeries(num_years, geo_mean_bond_return)
cash = base_series.StaticSeries(num_years, geo_mean_cash_return)

In [10]:
assets = [equity, bond, cash]
balance = initial_balance
glidepath = constants.fidelity_glidepath

In [11]:
portfolio = pf.Portfolio(assets, glidepath, balance)

# Inflation Scenario

In [69]:
inflation_scenario = base_series.StaticSeries(num_years, geo_mean_inflation)

In [70]:
cpi = base_series.StaticGrowthSeries(initial_value=100, growth_series=inflation)

# Consumption Scenario

In [72]:
consumption = StaticRetirementConsumption(retirement_consumption, cpi, initial_age, retirement_age)

In [73]:
deferral = StaticPreRetirementDeferral(initial_deferral, cpi, initial_age, retirement_age)

# Ret Analysis

In [74]:
retirement_analysis = ra.RetirementAnalysis(
    initial_age=initial_age,
    retirement_age=retirement_age,
    death_age=death_age,
    portfolio=portfolio,
    deferral_scenario=deferral,
    consumption_scenario=consumption,
    inflation_scenario=inflation,
    ssb_scenario=ssb)

NameError: name 'deferral_scenario' is not defined